## Importando as bibliotecas

In [1]:
import pandas as pd
import numpy as np
import requests

## Importando as funções que serão utilizadas no decorrer do notebook.

In [2]:
from nba_api.stats.endpoints import leaguedashteamstats
from nba_api.stats.endpoints import commonplayoffseries
from nba_api.stats.static import teams

--------------------------------------------------------------------------------

# Organizando os parâmetros que serão utilizados.


---

## 1. Os "Four Factors" (Eficiência)
*Os quatro pilares estatísticos que correlacionam mais forte com vitórias.*

* **eFG% (Effective Field Goal Percentage)**
    * *O que mede:* Eficiência de arremesso ajustada, onde bolas de 3 valem 50% a mais.
    * *Fórmula:* $$eFG\% = \frac{FGM + 0.5 \times 3PM}{FGA}$$

* **TOV% (Turnover Percentage)**
    * *O que mede:* Desperdício de bola a cada 100 posses.
    * *Fórmula:* $$TOV\% = \frac{TOV}{Possessions} \times 100$$

* **OREB% (Offensive Rebound Percentage)**
    * *O que mede:* Porcentagem de rebotes ofensivos capturados das oportunidades disponíveis.
    * *Fórmula:* $$OREB\% = \frac{OREB}{OREB + Opp\_DREB}$$

* **FTR (Free Throw Rate)**
    * *O que mede:* Capacidade de gerar pontos fáceis na linha de lance livre.
    * *Fórmula:* $$FTR = \frac{FTA}{FGA}$$

---

## 2. Estilo de Jogo e Identidade
*Métricas que definem "como" o time joga, independente de ganhar ou perder.*

* **Pace (Ritmo)**
    * *Definição:* Estimativa do número de posses de bola por 48 minutos.
    * *Interpretação:* Alto (>100) = Jogo rápido/transição. Baixo (<96) = Jogo de meia-quadra/cadenciado.

* **3PAr (3-Point Attempt Rate)**
    * *O que mede:* Preferência pelo arremesso de longa distância.
    * *Fórmula:* $$3PAr = \frac{3PA}{FGA}$$

* **Rim Frequency (% FGA at Rim)**
    * *O que mede:* Agressividade em atacar a cesta (arremessos dentro de 0-4 pés).
    * *Fórmula:* $$\frac{FGA_{Rim}}{FGA_{Total}}$$

---

## 3. Desempenho Geral (Ratings)
*Métricas normalizadas por 100 posses de bola.*

* **Offensive Rating (Ortg):** Pontos marcados por 100 posses.
* **Defensive Rating (Drtg):** Pontos sofridos por 100 posses.
* **Net Rating:** $$Ortg - Drtg$$ (Diferencial de pontos por 100 posses).

---

## 4. Coletividade e Caos Defensivo
*Playmaking e criação de eventos defensivos.*

* **AST% (Assist Percentage)**
    * *O que mede:* Porcentagem de cestas de quadra que foram assistidas. Indica circulação de bola vs. *Hero Ball*.

* **STL% (Steal Percentage)**
    * *O que mede:* Porcentagem de posses adversárias que terminam em roubo de bola.

* **BLK% (Block Percentage)**
    * *O que mede:* Porcentagem de arremessos de 2 pontos do adversário que foram bloqueados.

* **DREB% (Defensive Rebound Percentage)**
    * *O que mede:* Capacidade de encerrar a posse defensiva garantindo o rebote.

---

## 5. Construção de Elenco (Roster Metrics)
*Métricas focadas na distribuição de responsabilidade.*

* **Heliocentrism (Top 2 Usage Avg)**
    * *Definição:* A média da `USG%` dos dois jogadores com maior uso no time.
    * *Objetivo:* Identificar dependência excessiva de superestrelas.

* **Bench Points %**
    * *Definição:* Porcentagem dos pontos totais do time marcados por reservas.
    * *Objetivo:* Medir profundidade do elenco.

-------------------------------------------------------------------------------------------------------------------------------------------------------------

# Fazendo o Dataframe inicial dos ultimos 10 campeões
### Primeiramente colocarei, além da temporada, o 'full_name' do time, a temporada e o ID de cada time.

In [16]:
seasons = ["2015-16","2016-17","2017-18","2018-19","2019-20","2020-21",
           "2021-22","2022-23","2023-24","2024-25"]

teams_Series = ["CLEVELAND CAVALIERS", "GOLDEN STATE WARRIORS",
                      "GOLDEN STATE WARRIORS", "TORONTO RAPTORS","LOS ANGELES LAKERS",
                      "MILWAUKEE BUCKS", "GOLDEN STATE WARRIORS", "BOSTON CELTICS",
                        "DENVER NUGGETS", "OKLAHOMA CITY THUNDER"]

indexs = ["2016_CLE", "2017_GSW", "2018_GSW", "2019_TOR", "2020_LAL", "2021_MIL", "2022_GSW", "2023_BOS", "2024_DEN", "2025_OKC"]

df_last_10_champions = pd.DataFrame(index=indexs)
df_last_10_champions["TEAMS"] = teams_Series
df_last_10_champions["SEASON"] = seasons
df_last_10_champions

,TEAMS,SEASON
2016_CLE,CLEVELAND CAVALIERS,2015-16
2017_GSW,GOLDEN STATE WARRIORS,2016-17
2018_GSW,GOLDEN STATE WARRIORS,2017-18
2019_TOR,TORONTO RAPTORS,2018-19
2020_LAL,LOS ANGELES LAKERS,2019-20
2021_MIL,MILWAUKEE BUCKS,2020-21
2022_GSW,GOLDEN STATE WARRIORS,2021-22
2023_BOS,BOSTON CELTICS,2022-23
2024_DEN,DENVER NUGGETS,2023-24
2025_OKC,OKLAHOMA CITY THUNDER,2024-25


In [9]:
def find_ID_by_name(team_name):
    team_infos = teams.find_teams_by_full_name(team_name)
    return team_infos[0]["id"]

def find_abb_by_name(team_name):
    team_infos = teams.find_teams_by_full_name(team_name)
    return team_infos[0]["abbreviation"]

def getting_ID_row_by_name(teams_Series, df_teams):
    dict_times_ids = {}
    for time in teams_Series:
        dict_times_ids[time] = find_ID_by_name(time)

    for time in df_teams["TEAMS"]:
        for k in dict_times_ids.keys():
            if time == k:
                df_teams.loc[df_teams["TEAMS"] == time, "TEAM_ID"] = str(dict_times_ids[time])

In [5]:
# Adicionando a coluna ID no df_last_ten_champions
getting_ID_row_by_name(df_last_10_champions["TEAMS"], df_last_10_champions)
df_last_10_champions

,TEAMS,SEASON,TEAM_ID
0,CLEVELAND CAVALIERS,2015-16,1610612739
1,GOLDEN STATE WARRIORS,2016-17,1610612744
2,GOLDEN STATE WARRIORS,2017-18,1610612744
3,TORONTO RAPTORS,2018-19,1610612761
4,LOS ANGELES LAKERS,2019-20,1610612747
5,MILWAUKEE BUCKS,2020-21,1610612749
6,GOLDEN STATE WARRIORS,2021-22,1610612744
7,BOSTON CELTICS,2022-23,1610612738
8,DENVER NUGGETS,2023-24,1610612743
9,OKLAHOMA CITY THUNDER,2024-25,1610612760


# Será feito os datasets das métricas da liga em cada temporada

In [ ]:
nba_teams = [
    # Conferência Leste
    "ATLANTA HAWKS", "BOSTON CELTICS", "BROOKLYN NETS", "CHARLOTTE HORNETS", 
    "CHICAGO BULLS", "CLEVELAND CAVALIERS", "DETROIT PISTONS", "INDIANA PACERS", 
    "MIAMI HEAT", "MILWAUKEE BUCKS", "NEW YORK KNICKS", "ORLANDO MAGIC", 
    "PHILADELPHIA 76ERS", "TORONTO RAPTORS", "WASHINGTON WIZARDS",
    
    # Conferência Oeste
    "DALLAS MAVERICKS", "DENVER NUGGETS", "GOLDEN STATE WARRIORS", "HOUSTON ROCKETS", 
    "LOS ANGELES CLIPPERS", "LOS ANGELES LAKERS", "MEMPHIS GRIZZLIES", 
    "MINNESOTA TIMBERWOLVES", "NEW ORLEANS PELICANS", "OKLAHOMA CITY THUNDER", 
    "PHOENIX SUNS", "PORTLAND TRAIL BLAZERS", "SACRAMENTO KINGS", 
    "SAN ANTONIO SPURS", "UTAH JAZZ"
]

df_teams = pd.DataFrame()
df_teams["TEAMS"] = nba_teams

getting_ID_row_by_name(df_teams["TEAMS"], df_teams)
df_teams


----------------------------------------------------------------------------------------------------------------------

## Para fazer de cada season:

In [7]:
#EX.: 2017
indexs = []
for x in nba_teams:
    indexs.append(find_abb_by_name(x))
indexs_2017 = ['2017_' + item for item in indexs]
indexs_2017

['2017_ATL',
 '2017_BOS',
 '2017_BKN',
 '2017_CHA',
 '2017_CHI',
 '2017_CLE',
 '2017_DET',
 '2017_IND',
 '2017_MIA',
 '2017_MIL',
 '2017_NYK',
 '2017_ORL',
 '2017_PHI',
 '2017_TOR',
 '2017_WAS',
 '2017_DAL',
 '2017_DEN',
 '2017_GSW',
 '2017_HOU',
 '2017_LAC',
 '2017_LAL',
 '2017_MEM',
 '2017_MIN',
 '2017_NOP',
 '2017_OKC',
 '2017_PHX',
 '2017_POR',
 '2017_SAC',
 '2017_SAS',
 '2017_UTA']

In [10]:
df_2017 = pd.DataFrame(index = indexs_2017)
df_2017["TEAMS"] = nba_teams
getting_ID_row_by_name(df_2017["TEAMS"], df_2017)
df_2017 

,TEAMS,TEAM_ID
2017_ATL,ATLANTA HAWKS,1610612737
2017_BOS,BOSTON CELTICS,1610612738
2017_BKN,BROOKLYN NETS,1610612751
2017_CHA,CHARLOTTE HORNETS,1610612766
2017_CHI,CHICAGO BULLS,1610612741
2017_CLE,CLEVELAND CAVALIERS,1610612739
2017_DET,DETROIT PISTONS,1610612765
2017_IND,INDIANA PACERS,1610612754
2017_MIA,MIAMI HEAT,1610612748
2017_MIL,MILWAUKEE BUCKS,1610612749


### Buscando os FOUR FACTORS